In [1]:
# Run only once to install new version of solo_epd_loader
#%pip install --upgrade solo_epd_loader
#%pip install solo_epd_loader==0.2.8
#%pip install --upgrade sunpy-soar

In [2]:
from epd_plot_shift import *
from run_the_fit import *
#from solo_epd_loader import calc_electrons
from solo_epd_loader import epd_load, calc_electrons, combine_pixels


/home/annafed/anaconda3/envs/annamaria/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Notebook to plot and fit SolO EPD SEE data
#### General information about the notebook:
This Notebook allows the user to load, plot and fit SolO EPD SEE energy data.
The data loading process uses solo-epd-loader. 
<br>This package can be installed simply by running   %pip install git+https://github.com/jgieseler/solo-epd-loader    in any cell of the notebook. Note that this should only be done once. The package can be updated by running: %pip install git+https://github.com/jgieseler/solo-epd-loader --upgrade
<br>The data will be saved as cdf files to a local folder provided by the user. The data is then used to create dataframes of the electron data for each instrument and energy channel. 
<br>The flux data will be plotted as a timeseries. The timeseries include all the energy channels for each instrument. (add  expl pa) A flux peak will be searchered for each energy channel using the provided injection time and following the velocity dispersion. These values will then be plotted against the energy corresponding to the channels to form a spectrum. 
<br>The spectrum can then be fit with various functions to analyse its shape.


### NECESSARY INPUTS
Choose a folder where you wish to save the data from the loading, plotting and fitting process. 
If create_sub_folders = True, a new folder will be created for each event and the data etc will be saved there. 
<br> date : (datetime) format: datetime(yyyy, mm, dd, HH, MM) The date and approximate injection time of the event.

In [3]:
path =  r'/home/annafed/Documents/STIX_EPD_event_analysis/STEP_EPD_comparison/'
create_sub_folders = True
threshold_folders = False # if true saving to separate folders does not work. 
plot_n_data_folders = True
save_fit_run_variables_to_separate_folders = True

date = datetime(2020, 12, 10, 23, 46) #onset time or event ID inj time specified later
saving = True
pixels = True

position: this variable contains information about the location of SolO on the day of the event as well as the travel time of light and electrons with energy corresponding to the lowest STEP energy channel (4 keV) and the highest HET energy channel (10 MeV).

In [4]:
position = position_and_traveltime(date)

INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
-------------------------------------------  -----  ---------
Distance of SolO from the Sun                [AU]    0.82
Length of the Parker Spiral for 400 km/s sw  [AU]    0.909607
Travel time of 4 KeV electrons               [min]  60.8137
Travel time of 10 MeV electrons              [min]   7.57392
Travel time of light                         [min]   6.81973
-------------------------------------------  -----  ---------


### INPUTS FOR PLOTTING AND FITTING

Choose a viewing direction and time averaging for the data.
<br> __direction__ (string): 'sun', 'asun', 'north', 'south'    NOTE: STEP has 'sun' direction only. 
<br> __averaging__(str):  data averaging

<br> Possible values:
<br>‘W’
<br>‘D’ / ‘days’ / ‘day’
<br>‘hours’ / ‘hour’ / ‘hr’ / ‘h’
<br>‘m’ / ‘minute’ / ‘min’ / ‘minutes’ / ‘T’
<br>‘S’ / ‘seconds’ / ‘sec’ / ‘second’
<br>‘ms’ / ‘milliseconds’ / ‘millisecond’ / ‘milli’ / ‘millis’ / ‘L’
<br>‘us’ / ‘microseconds’ / ‘microsecond’ / ‘micro’ / ‘micros’ / ‘U’
<br>‘ns’ / ‘nanoseconds’ / ‘nano’ / ‘nanos’ / ‘nanosecond’ / ‘N’

<br> __ion_correction__ (bool): if True will use ion corrected EPT data (use only if inclusing EPT data)
<br> __bg_subtraction__ (bool): if True will use background subtracted data (else choose False)
<br> __step__ (bool): if True step data will be dowloaded and plotted
<br> __ept__ (bool): if True ept data will be dowloaded and plotted
<br> __het__ (bool): if True het data will be dowloaded and plotted
<br> __fit_type__ (string): the combination of instriment data that will be fit. Options: 'step', 'ept', 'het', 'step_ept', 'ept_het', 'step_ept_het'
<br> __which_fit__ (string): the type of pl function that will be fit to the data. Note that if the data cannot be fit with a specific function the software will choose the next best option. Options: 
<br> __'single'__ will force a single pl fit to the data.
<br> __'broken'__ will force a broken pl fit to the data but ONLY if the break point is within the energy range otherwise a sigle pl fit will be produced instead.
<br> __'best_sb'__ will choose automatically the best fit type between single and broken by comparing the redchis of the fits.
<br> __'cut'__ will produce a single pl fit with an exponential cutoff point. If the cutoff point is outside of the energy range a broken or single pl will be fit instead.
<br> __'broken_cut'__ will produce a broken pl fit with an exponential cutoff point. If the cutoff point is outside of the energy range a broken, cut or single pl will be fit instead.
<br> __'best_cb'__ will choose automatically the best fit type between broken and cut by comparing the redchis of the fits. A single pl will be fit to the data if the other fits are not physical. 
<br> __'triple'__ will try to fit a triple pl to the data. If this is not possible, the next best option will be fit to the data. 
<br> __'best'__ will automatically choose the best out of all the fitting options.

In [5]:
direction = 'sun'
averaging = '1min'
contamination_threshold = None  # either None or an integer >0 (STEP)

ion_correction = True 
bg_subtraction = True

step = True
ept = True
het = True

fit_type= 'step_ept'
which_fit = 'broken'


### NON-NECESSARY INPUTS

#### PLOTTING TIMESERIES AND SPECTRUM (NO FIT)
The following inputs can be changed, but the notebook will run without any furher changes.
<br> __start_date__ and __end_date__ (int or datetime): The start and end date to download data. These should be different only in case the event occurs close to midnight. Automatically done if the event is less that 4h from midnight. Format: datetime(yyyy, mm, dd) or yyyymmdd.
<br> __plot_start__ and __plot_end__ (string): The start and end time for the timeseries plot. Automatically taken as $\pm$4h. Format: 'yyyy-mm-dd-HHMM'
<br> __t_inj__ (string): taken automatically from previously defined value (date). Also this value is changed by using the light travel time (position) to fit the peak into the search window. This value is fixed to the highest HET channel (close traveltime to light tt for electrons), the rest of the window times are calculated using velocity dispersion.
<br> For the background there are two options: either a fixed bg window that starts and ends at the same time for each energy channel or a bg window that follows the peak search window (beacuse of velocity dispersion at different times for different energy channels). 
For a fixed bg window: __bg_start__ and __bg_end__ (string, defaults to None) 'yyyy-mm-dd-HHMM'
For a bg window that follows the peaxk search window: __bg_distance from window__ (int, defauts to 120): the distance of the start time of the bg window from the start of the search window in minutes. __bg_period__ (int, defaults to 60) the length of the bg window in minutes.
<br> __travel_distance__ (float): Length of the Parker Spiral in AU. This is taken automatically from the position variable.
<br> __travel_distance_second_slope__ (float, defaults to None): __travel_distance_second_slope__ > __travel_distance__ If specified this will create a peak search window that broadens at lower energies. If specified set fixed_window = None.
<br> __fixed_window__ (int, defauts to 60): the length of the search window in minutes. 
<br> __autodownload__ (bool, defaults to True): If True will automatically download the data to analyse the event.
<br> __data_type__ (string, defaults to 'l2'): 'll' = low latency, 'l1' = level 1 data, 'l2' = level 2 data for scientific research.
<br> __plot_pa__ (bool, defauts to True): plots the pitch angle coverage for the event.

In [6]:
startdate = date-timedelta(hours = 6)
start_date = startdate.date()
enddate = date+timedelta(hours = 6)
end_date = enddate.date()

plot_start = str(date-timedelta(hours = 6))[:-3].replace(' ', '-').replace(':', '')
plot_end = str(date+timedelta(hours = 6))[:-3].replace(' ', '-').replace(':', '')


t_inj = str(date)[:-3].replace(' ', '-').replace(':', '')
date_folder = t_inj
#plot_path = path_to_folder
#if create_sub_folders:
#  create_new_path(path, t_inj)#, threshold_folders = threshold_folders, contamination_threshold = contamination_threshold, plots_n_data = plot_n_data_folders)
#  path_to_folder = path+t_inj+'/'
# plot_path = path_to_folder

if contamination_threshold == None:
    threshold_folders = False
if create_sub_folders:
    create_new_path(path, t_inj, threshold_folders = threshold_folders, contamination_threshold = contamination_threshold, plots_n_data = plot_n_data_folders)
    path_to_folder = path+t_inj+'/'
    plot_path = path_to_folder
if threshold_folders:
    path_to_folder = path_to_folder+'contamination_threshold_'+str(contamination_threshold)+'/'
if plot_n_data_folders:
    plot_path = path_to_folder+'plots'
 
    
# Need to subtract the light travel time at SolO distance because the Raul inj. times
# in the STIX/EPD table are shifted from sun to SolO and I need inj. time at the Sun
#t_inj = str(date-timedelta(minutes = position[4][2]))[:-3].replace(' ', '-').replace(':', '')
#t_inj = str(date-timedelta(minutes = 20))[:-3].replace(' ', '-').replace(':', '')

#print(date_folder)

#bg_start = None 
#bg_end = None 
#bg_distance_from_window = '10min'
#bg_period = '3h'

#travel_distance = position[0][2]
#travel_distance_second_slope = None
#fixed_window = '60min' #either input time period in minutes or None

autodownload_data = True
data_type = 'l2'
plot_pa = True


Creating new directory /home/annafed/Documents/STIX_EPD_event_analysis/STEP_EPD_comparison/2020-12-10-2346


#### INPUTS FOR THE FIT
<br> __c1_guess__ (int or float, defaults to 1000): the flux value corresponding to 0.1 MeV
<br> __g1_guess__ (int or float, defaults to -1.9): the slope of a single pl or the first part of a broken or triple pl. __g1_guess__ > __g2_guess__, __g3_guess__ and __g1_guess__< 0.
<br> __g2_guess__ (int or float, defaults to -2.5): the slope of the broken pl after the break or the slope between the two breaks for a triple pl. __g1_guess__> __g2_guess__ >  __g3_guess__ and __g2_guess__< 0.
<br> __g3_guess__ (int or float, defaults to -4): the slope of a triple pl after the second break. __g1_guess__, __g2_guess__ >  __g3_guess__ and __g3_guess__< 0.
<br> __alpha_guess__ and __beta_guess__ (int or float, defaults to 10): the smoothness of the first and second break respectively.
<br> __break_guess_low__ (int or float, defaults to 0.6 MeV): the energy corresponding to the first break in the spectrum (in MeV).
<br>__break_guess_high__ (int or float, defaults to 1.2 MeV): the energy corresponding to the second break in the spectrum (in MeV).
<br> __cut_guess__ (int or float, defaults to 1.2 MeV): the energy corresponding to the exponential cutoff in the spectrum (in MeV).
<br> __fit_to__ (string, defaults to 'peak'): 'peak' or 'average'
<br> __e_min__ and __e_max__ (int or float, defaults to None): minimum and maximum energy to fit. If None will automatically be taken as min and max fittable values. step energy range: 0.004323343613-0.07803193193 MeV, ept energy range: 0.03295087252-0.452730295 MeV, het energy range: 0.6859485403-10.62300288 MeV, second het channel: 1.590048112 MeV
<br> __use_random__ (bool, defaults to True): if True the software will choose random values close to the guess values that will be fit to the data to find the fit with the lowest redchisq.
<br> __random_iterations__ (int, defaults to 20): number of times the random values will be chosen.
<br> __sigma__ (int, defaults to 3.): standard deviation from the background.
<br> __rel_err__ (float, defaults to 0.5): The absolute value of the uncertainty of the bg subtracted flux peak divided by the bg subtracted peak.
<br> __frac_nan_threshold__ (float, defaults to 0.9): exclude all spectrum values with fraction of non-nan data points in search window lower than the corresponding value.
<br> __leave_out_1st_het_chan__ (bool, defaults to True): If True, the first HET channel will be left out from the fit.
<br> __shift_step_data__ (bool, defaults to False): If True, STEP data will be shifted (up or down, intensity wise) by a factor equal to shift_factor.
<br> __shift_factor__ (float, defaults to None): Factor to shift STEP data (e.g. 0.8). 
<br> __legend_details__ (bool, optional): If True, the final fit type and the reduced chi square will be dislayed in the legend.
	


In [7]:
c1_guess = 1000
g1_guess = -1.9
g2_guess = -2.5
g3_guess = -4
alpha_guess = 10
beta_guess = 10
break_guess_low = 0.6
break_guess_high = 1.2
cut_guess = 1.2

fit_to = 'peak'

e_min = None
e_max = None

use_random = True
random_iterations = 20

if saving:
    save_fig = True
    save_fit_variables = True
    save_fitrun = True

sigma = 3
rel_err = 0.5
frac_nan_threshold = 0.1
leave_out_1st_het_chan = False
shift_step_data = False
shift_factor = None #0.8

detailed_legend = True
only_averages = False

## LOAD DATA

In [8]:
if plot_pa:
    msdate = dt.datetime.combine(pd.to_datetime(plot_start).date(), dt.time.min)
    medate = dt.datetime.combine(pd.to_datetime(plot_end).date()+ dt.timedelta(days=1), dt.time.min) #+ dt.timedelta(days=1)

    mag_srf = solo_mag_loader(msdate, medate, level='l2', frame='srf', av=averaging, path=path)
    

Loading MAG...


In [9]:
if step:   
    # read in STEP original data to df_particles_step_org. electron fluxes calculated later with calc_electrons
    df_particles_step_org, energies_step = epd_load(sensor='step', level='l2', startdate=start_date, 
                                                    enddate=end_date, path=path, autodownload=True, 
                                                    only_averages=only_averages)
    
    

Loading /home/annafed/Documents/STIX_EPD_event_analysis/STEP_EPD_comparison/l2/epd/step/solo_L2_epd-step-rates_20201210_V01.cdf
Loading /home/annafed/Documents/STIX_EPD_event_analysis/STEP_EPD_comparison/l2/epd/step/solo_L2_epd-step-rates_20201211_V01.cdf


In [10]:
#df_particles_step

In [11]:
if ept:
    df_protons_ept, df_electrons_ept, energies_ept = epd_load(sensor = 'ept', viewing = direction, level = 'l2',
                                                                  startdate = start_date, enddate = end_date, 
                                                                  path=path, autodownload=True)
    

In [12]:
if het:  
    df_protons_het, df_electrons_het, energies_het = epd_load(sensor = 'het', viewing = direction, level = 'l2',
                                                            startdate = start_date, enddate = end_date, 
                                                            path=path, autodownload=True)
   

In [13]:
# change depending on onset time difference 
t_inj = str(date-timedelta(minutes = 20))[:-3].replace(' ', '-').replace(':', '')

print(date_folder)

bg_start = None 
bg_end = None 
bg_distance_from_window = '20min'
bg_period = '3h'

travel_distance = position[0][2]
travel_distance_second_slope = None
fixed_window = '2h' #either input time period in minutes or None


2020-12-10-2346


In [14]:
date_string = str(date.date())
#plot_pa = False
#coverage = None


In [21]:
#write_to_csv(data_step_pix, date = date_string, path=path_to_folder, direction = direction, centre_pix = centre_pix)
for i in df_particles_step_pix.columns: print(i)

DELTA_EPOCH
Integral_Avg_Uncertainty_0
Integral_Avg_Uncertainty_1
Integral_Avg_Uncertainty_2
Integral_Avg_Uncertainty_3
Integral_Avg_Uncertainty_4
Integral_Avg_Uncertainty_5
Integral_Avg_Uncertainty_6
Integral_Avg_Uncertainty_7
Integral_Avg_Uncertainty_8
Integral_Avg_Uncertainty_9
Integral_Avg_Uncertainty_10
Integral_Avg_Uncertainty_11
Integral_Avg_Uncertainty_12
Integral_Avg_Uncertainty_13
Integral_Avg_Uncertainty_14
Integral_Avg_Uncertainty_15
Integral_Avg_Uncertainty_16
Integral_Avg_Uncertainty_17
Integral_Avg_Uncertainty_18
Integral_Avg_Uncertainty_19
Integral_Avg_Uncertainty_20
Integral_Avg_Uncertainty_21
Integral_Avg_Uncertainty_22
Integral_Avg_Uncertainty_23
Integral_Avg_Uncertainty_24
Integral_Avg_Uncertainty_25
Integral_Avg_Uncertainty_26
Integral_Avg_Uncertainty_27
Integral_Avg_Uncertainty_28
Integral_Avg_Uncertainty_29
Integral_Avg_Uncertainty_30
Integral_Avg_Uncertainty_31
Integral_Avg_Uncertainty_32
Integral_Avg_Uncertainty_33
Integral_Avg_Uncertainty_34
Integral_Avg_Uncer

## STEP

In [16]:
centre_pix = True
if step and pixels:
    # calculate resampled STEP electron fluxes (+ ions)
    df_particles_step_pix = calc_electrons(df=df_particles_step_org, meta=energies_step, 
                                       contamination_threshold=contamination_threshold, only_averages=only_averages,
                                       resample=averaging)
    # should be an if statement. So if we only want to use certaint pixels then this
    #if pixels:
    df_particles_step_pix = combine_pixels(df=df_particles_step_pix, meta=energies_step, pixels=['02', '03', '04', '07', '08', '09', '12', '13', '14'])
    
    data_step_pix = extract_electron_data(df_particles_step_pix, energies_step, plot_start, plot_end, t_inj, 
                                      bgstart = bg_start , bgend = bg_end, bg_distance_from_window = bg_distance_from_window, 
                                      bg_period = bg_period, travel_distance = travel_distance, 
                                      travel_distance_second_slope = travel_distance_second_slope, fixed_window = fixed_window, 
                                      instrument = 'step', data_type = 'l2', averaging=averaging, 
                                      masking=False,  centre_pix = centre_pix)
    if plot_pa:
        if averaging is not None:
            mag_srf = mag_srf.resample(averaging).mean()
        coverage = calc_pa_coverage('step', mag_srf)

    if saving: write_to_csv(data_step_pix, date = date_string, path=path_to_folder, direction = direction, centre_pix = centre_pix)
    plot_channels(data_step_pix, bg_subtraction=False, path=plot_path, savefig=saving, plot_pa=plot_pa, coverage=coverage, sensor = 'step', sigma = sigma, frac_nan_threshold = frac_nan_threshold, rel_err_threshold = rel_err, centre_pix = centre_pix)
    bg_b = bg_subtraction
    plot_channels(data_step_pix, bg_subtraction=bg_b, path=plot_path, savefig=saving, plot_pa=plot_pa, coverage=coverage, sensor = 'step', sigma = sigma, frac_nan_threshold = frac_nan_threshold, rel_err_threshold = rel_err, centre_pix = centre_pix)
    bg_b = bg_subtraction
    plot_spectrum_peak(data_step_pix, bg_subtraction=bg_b, path=plot_path, savefig=saving, sigma = sigma, frac_nan_threshold = frac_nan_threshold, rel_err_threshold= rel_err, direction = direction, centre_pix = centre_pix)
    #plot_spectrum_average(data_step, bg_subtraction=bg_b, savefig=saving, path=path_to_folder, sigma=sigma, frac_nan_threshold=frac_nan_threshold, rel_err_threshold=rel_err, direction=direction)
    


100%|██████████| 336/336 [00:01<00:00, 206.76it/s]


Info: contamination_threshold will only be applied if it is an integer. Otherwise only negative fluxes are removed.


KeyError: 'Electron_Comb_Flux_8'

In [ ]:
centre_pix = False
if step:
    # calculate resampled STEP electron fluxes (+ ions)
    df_particles_step = calc_electrons(df=df_particles_step_org, meta=energies_step, 
                                       contamination_threshold=contamination_threshold, only_averages=True,
                                       resample=averaging)
    # should be an if statement. So if we only want to use certaint pixels then this
    #if pixels:
    #df_particles_step = combine_pixels(df=df_particles_step_org, meta=energies_step, pixels=['02', '03', '04', '07', '08', '09', '12', '13', '14'])
    
    data_step = extract_electron_data(df_particles_step, energies_step, plot_start, plot_end, t_inj, 
                                      bgstart = bg_start , bgend = bg_end, bg_distance_from_window = bg_distance_from_window, 
                                      bg_period = bg_period, travel_distance = travel_distance, 
                                      travel_distance_second_slope = travel_distance_second_slope, fixed_window = fixed_window, 
                                      instrument = 'step', data_type = 'l2', averaging=averaging, 
                                      masking=False,  centre_pix = centre_pix)
    if plot_pa:
        if averaging is not None:
            mag_srf = mag_srf.resample(averaging).mean()
        coverage = calc_pa_coverage('step', mag_srf)

    if saving: write_to_csv(data_step, date = date_string, path=path_to_folder, direction = direction)
    plot_channels(data_step, bg_subtraction=False, path=plot_path, savefig=saving, plot_pa=plot_pa, coverage=coverage, sensor = 'step', sigma = sigma, frac_nan_threshold = frac_nan_threshold, rel_err_threshold = rel_err, centre_pix = centre_pix)
    bg_b = bg_subtraction
    plot_channels(data_step, bg_subtraction=bg_b, path=plot_path, savefig=saving, plot_pa=plot_pa, coverage=coverage, sensor = 'step', sigma = sigma, frac_nan_threshold = frac_nan_threshold, rel_err_threshold = rel_err, centre_pix = centre_pix)
    bg_b = bg_subtraction
    plot_spectrum_peak(data_step, bg_subtraction=bg_b, path=plot_path, savefig=saving, sigma = sigma, frac_nan_threshold = frac_nan_threshold, rel_err_threshold= rel_err, direction = direction, centre_pix = centre_pix)
    #plot_spectrum_average(data_step, bg_subtraction=bg_b, savefig=saving, path=path_to_folder, sigma=sigma, frac_nan_threshold=frac_nan_threshold, rel_err_threshold=rel_err, direction=direction)
    


In [ ]:
data_step


In [ ]:
ceter_pix_average_comparison_spec(data_step, data_step_pix, bg_subtraction=bg_b, path=plot_path, savefig=saving, sigma = sigma, frac_nan_threshold = frac_nan_threshold, rel_err_threshold= rel_err, direction = direction)
    

## EPT

In [ ]:
if ept:
    ion_b = ion_correction    
    data_ept = extract_electron_data(df_electrons_ept, energies_ept, plot_start, plot_end, t_inj, bgstart = bg_start, 
                                     bgend = bg_end, bg_distance_from_window = bg_distance_from_window, bg_period = bg_period, 
                                     travel_distance = travel_distance, travel_distance_second_slope=travel_distance_second_slope, 
                                     fixed_window=fixed_window, instrument = 'ept', data_type = 'l2', averaging=averaging, masking=True, 
                                     ion_conta_corr=ion_b, df_protons = df_protons_ept)

    
    if plot_pa:
        mag_srf = mag_srf.resample(averaging).mean()
        coverage = calc_pa_coverage('ept', mag_srf)
    else:
        coverage = None

    if saving: write_to_csv(data_ept, date = date_string, path=path_to_folder, direction = direction)
    plot_channels(data_ept, bg_subtraction=False, path=plot_path, savefig=saving, plot_pa=plot_pa, coverage=coverage, sensor = 'ept', sigma = sigma, frac_nan_threshold = frac_nan_threshold, rel_err_threshold = rel_err, viewing = direction)
    bg_b = bg_subtraction
    plot_channels(data_ept, bg_subtraction=bg_b, path=plot_path, savefig=saving, plot_pa=plot_pa, coverage=coverage, sensor = 'ept', sigma = sigma, frac_nan_threshold = frac_nan_threshold, rel_err_threshold = rel_err, viewing = direction)
    bg_b = bg_subtraction
    plot_spectrum_peak(data_ept, bg_subtraction=bg_b, path=plot_path, savefig=saving, sigma = sigma, frac_nan_threshold = frac_nan_threshold, rel_err_threshold = rel_err, direction = direction) 
    #plot_spectrum_average(data_ept, bg_subtraction=bg_b, path=path_to_folder, savefig=saving, frac_nan_threshold=frac_nan_threshold)      


## HET

In [ ]:
if het:
    data_het = extract_electron_data(df_electrons_het, energies_het, plot_start, plot_end, t_inj, 
                                     bgstart = bg_start, bgend = bg_end,  bg_distance_from_window = bg_distance_from_window,
                                     bg_period = bg_period, travel_distance = travel_distance, 
                                     travel_distance_second_slope = travel_distance_second_slope, 
                                     fixed_window = fixed_window, instrument='het', data_type='l2', 
                                     averaging=averaging, df_protons = df_protons_het)

    if plot_pa:
        mag_srf = mag_srf.resample(averaging).mean()
        coverage = calc_pa_coverage('het', mag_srf)

    if saving: write_to_csv(data_het, date = date_string, path=path_to_folder, direction = direction)
    plot_channels(data_het, bg_subtraction=False, path=plot_path, savefig=saving, plot_pa=plot_pa, coverage=coverage, sensor = 'het', sigma = sigma, frac_nan_threshold = frac_nan_threshold, rel_err_threshold = rel_err, viewing = direction)
    bg_b = bg_subtraction
    plot_channels(data_het, bg_subtraction=bg_b, path=plot_path, savefig=saving, plot_pa=plot_pa, coverage=coverage, sensor = 'het', sigma = sigma, frac_nan_threshold = frac_nan_threshold, rel_err_threshold = rel_err, viewing = direction)
    bg_b = bg_subtraction
    plot_spectrum_peak(data_het, bg_subtraction=bg_b, path=plot_path, savefig=saving, sigma = sigma, frac_nan_threshold = frac_nan_threshold, rel_err_threshold = rel_err, direction = direction)


## FIT

In [ ]:
c1_guess = 1000
g1_guess = -1.9
g2_guess = -2.5
g3_guess = -4.
alpha_guess = 10.
beta_guess = 10.
break_guess_low = 0.12
break_guess_high = 0.46
cut_guess = 0.12

fit_to = 'peak'

e_min = None
e_max = None
random_iterations= 20
#fit_type= 'ept'
which_fit = 'broken'
detailed_legend = True
centre_pix = True

In [ ]:
FIT_DATA(path_to_folder, date, averaging = averaging, fit_type= fit_type, step = step, ept = ept, het = het, direction = direction,
         which_fit = which_fit, sigma = sigma, rel_err = rel_err, frac_nan_threshold = frac_nan_threshold, fit_to = fit_to, 
         e_min = e_min, e_max = e_max, g1_guess = g1_guess, g2_guess = g2_guess, g3_guess = g3_guess, c1_guess = c1_guess, 
         alpha_guess = alpha_guess, beta_guess = beta_guess, break_guess_low = break_guess_low, break_guess_high = break_guess_high, cut_guess = cut_guess, 
         use_random = use_random, iterations = random_iterations, leave_out_1st_het_chan = leave_out_1st_het_chan, shift_step_data = shift_step_data, 
         shift_factor = shift_factor, save_fig = saving, save_pickle = False, save_fit_variables = saving, save_fitrun = saving, legend_details = detailed_legend, 
         ion_correction = ion_correction, bg_subtraction = bg_subtraction, fit_to_separate_folder = plot_n_data_folders,  centre_pix = centre_pix)


In [ ]:
FIT_DATA(path_to_folder, date, averaging = averaging, fit_type= fit_type, step = step, ept = ept, het = het, direction = direction,
         which_fit = which_fit, sigma = sigma, rel_err = rel_err, frac_nan_threshold = frac_nan_threshold, fit_to = fit_to, 
         e_min = e_min, e_max = e_max, g1_guess = g1_guess, g2_guess = g2_guess, g3_guess = g3_guess, c1_guess = c1_guess, 
         alpha_guess = alpha_guess, beta_guess = beta_guess, break_guess_low = break_guess_low, break_guess_high = break_guess_high, cut_guess = cut_guess, 
         use_random = use_random, iterations = random_iterations, leave_out_1st_het_chan = leave_out_1st_het_chan, shift_step_data = shift_step_data, 
         shift_factor = shift_factor, save_fig = saving, save_pickle = False, save_fit_variables = saving, save_fitrun = saving, legend_details = detailed_legend, 
         ion_correction = ion_correction, bg_subtraction = bg_subtraction, fit_to_separate_folder = plot_n_data_folders,  centre_pix = False)


In [ ]:
#if averaging<1.:
 #   averaging = str(int(averaging*60))+'s'...
#else:
 #   averaging = str(averaging)+'min'

if save_fit_run_variables_to_separate_folders:
    fit_var_file = date_folder+'-fit-result-variables_'+fit_type+'-'+fit_to+'-'+which_fit+'-l2-'+averaging+'-'+direction+'.csv'
    run_var_file = date_folder+'-all-fit-variables_'+fit_type+'-'+fit_to+'-'+which_fit+'-l2-'+averaging+'-'+direction+'.csv'

    save_fit_and_run_variables_to_separate_folders(path, date_folder, fit_var_file, run_var_file)
    

In [ ]:
import matplotlib.ticker as ticker

if bg_subtraction == True:
    bg_subt = 'on'
else:
    bg_subt = 'off'

data_ept_no_ion_corr = extract_electron_data(df_electrons_ept, energies_ept, plot_start, plot_end, t_inj, bgstart = bg_start, 
                                     bgend = bg_end, bg_distance_from_window = bg_distance_from_window, bg_period = bg_period, 
                                     travel_distance = travel_distance, travel_distance_second_slope=travel_distance_second_slope, 
                                     fixed_window=fixed_window, instrument = 'ept', data_type = 'l2', averaging=averaging, masking=True, 
                                     ion_conta_corr=False, df_protons = df_protons_ept)

write_to_csv(data_ept_no_ion_corr, date = date_string, path=path_to_folder, direction = direction)


file_path = path + date.strftime('%Y-%m-%d-%H%M') +'/'
os.getcwd()
os.chdir(file_path)
data_no_ion_corr = 'electron_data-'+ date.strftime('%Y-%m-%d') + '-EPT-' + direction + '-L2-' + averaging + '_averaging.csv'
data_ion_corr = 'electron_data-'+ date.strftime('%Y-%m-%d') + '-EPT-' + direction + '-L2-' + averaging+ '_averaging-ion_corr.csv'
df1 = pd.read_csv(data_no_ion_corr, sep=';')  
df2 = pd.read_csv(data_ion_corr, sep=';')

y1 = df1.Flux_peak
y2 = df2.Flux_peak
x = df1.Primary_energy
ratio = y1 / y2

fig = plt.figure(figsize=(10, 10))
gs = fig.add_gridspec(nrows=2, ncols=1, height_ratios=[1,1/3])

def color_plot(direction):
    color_dict = {'sun': 'crimson', 'asun': 'orange', 'north': 'darkslateblue', 'south': 'c'}
    color = color_dict.get(direction.lower(), '')
    ax1 = fig.add_subplot(gs[0])
    ax1.errorbar(df1.Primary_energy, df1.Flux_peak, df1.Energy_error_low, df1.Energy_error_high, color='k', fmt='o', label='Without ion contamination correction') 
    ax1.errorbar(df1.Primary_energy, df1.Background_flux, df1.rel_backsub_peak_err, color='k', fmt='o', alpha = 0.3, label='Background without ion contamination correction')
    ax1.errorbar(df2.Primary_energy, df2.Flux_peak, df2.Energy_error_low, df2.Energy_error_high, color=color, fmt='o', label='With ion contamination correction') 
    ax1.errorbar(df2.Primary_energy, df2.Background_flux, df2.rel_backsub_peak_err, color=color, fmt='o', alpha = 0.3, label = 'Background with ion contamination correction')

    ax1.set_title('EPT, ' + direction + ', ' + date.strftime('%Y-%m-%d') + ', ' + str(averaging) + ' averaging, bg subtraction ' + bg_subt, fontsize = 16)
    ax1.set_ylabel(r'Flux [1/s cm$^{2}$ sr MeV]', fontsize = 16)
    ax1.tick_params(axis='y', which='major', labelsize=14)
    ax1.legend(loc=0, fontsize = 12)
    ax1.set_yscale('log')
    ax1.set_xscale('log')

    ax2 = fig.add_subplot(gs[1])
    ax2.scatter(x, ratio, color=color, label='Electron flux / corrected electron flux')

    ax2.set_xlabel('Energy [MeV]', fontsize=16)
    ax2.set_ylabel('Flux ratio', fontsize=16)
    ax2.set_yscale('log')
    ax2.set_xscale('log')
    ax2.legend(loc=0, fontsize=12)

    ax2.tick_params(axis='x', which='minor', labelsize=12)
    ax2.tick_params(axis='x', which='major', labelsize=14)
    ax2.tick_params(axis='y', which='major', labelsize=14)
    ax2.tick_params(axis='y', which='minor', labelsize=12)
    ax2.xaxis.set_minor_formatter(FormatStrFormatter("%.2f"))

    minor_positions = [0.03, 0.04, 0.05, 0.06, 0.08, 0.20, 0.30, 0.40, 0.50]
    minor_locator = ticker.FixedLocator(minor_positions)
    ax2.xaxis.set_minor_locator(minor_locator)

color_plot(direction)

plt.subplots_adjust(hspace=0)
plt.savefig(path +date_folder+ '/plots/EPT_ion_corr_spectra_comparison-'+direction+'-'+averaging+'.png', dpi=300, bbox_inches='tight')
#folder = 
#name = 'Ion_contamination_EPT_' + direction + '_' + date.strftime('%Y-%m-%d') + '_' + str(averaging) + '_averaging.jpg'
#plt.savefig(os.path.join(folder, name), bbox_inches="tight")  

import matplotlib.ticker as ticker

data_ept_no_ion_corr = extract_electron_data(df_electrons_ept, energies_ept, plot_start, plot_end, t_inj, bgstart = bg_start, 
                                     bgend = bg_end, bg_distance_from_window = bg_distance_from_window, bg_period = bg_period, 
                                     travel_distance = travel_distance, travel_distance_second_slope=travel_distance_second_slope, 
                                     fixed_window=fixed_window, instrument = 'ept', data_type = 'l2', averaging=averaging, masking=True, 
                                     ion_conta_corr=False, df_protons = df_protons_ept)

write_to_csv(data_ept_no_ion_corr, date = date_string, path=path_to_folder, direction = direction)

file_path = path + date.strftime('%Y-%m-%d-%H%M') +'/'
os.getcwd()
os.chdir(file_path)
data_no_ion_corr = 'electron_data-'+ date.strftime('%Y-%m-%d') + '-EPT-' + direction + '-L2-' + averaging + '_averaging.csv'
data_ion_corr = 'electron_data-'+ date.strftime('%Y-%m-%d') + '-EPT-' + direction + '-L2-' + averaging+ '_averaging-ion_corr.csv'
df1 = pd.read_csv(data_no_ion_corr, sep=';')  
df2 = pd.read_csv(data_ion_corr, sep=';')

y1 = df1.Flux_peak
y2 = df2.Flux_peak
x = df1.Primary_energy
ratio = y1 / y2

fig = plt.figure(figsize=(10, 10))
gs = fig.add_gridspec(nrows=2, ncols=1, height_ratios=[1,1/3])

#def color_plot(direction):
color_dict = {'sun': 'crimson', 'asun': 'orange', 'north': 'darkslateblue', 'south': 'c'}
color = color_dict.get(direction.lower(), '')
ax1 = fig.add_subplot(gs[0])
ax1.errorbar(df1.Primary_energy, df1.Flux_peak, df1.Energy_error_low, df1.Energy_error_high, color='k', fmt='o', label='With ion contamination') 
ax1.errorbar(df1.Primary_energy, df1.Background_flux, df1.rel_backsub_peak_err, color='k', fmt='o', alpha = 0.3, label='Background with ion contamination')
ax1.errorbar(df2.Primary_energy, df2.Flux_peak, df2.Energy_error_low, df2.Energy_error_high, color=color, fmt='o', label='Without ion contamination') 
ax1.errorbar(df2.Primary_energy, df2.Background_flux, df2.rel_backsub_peak_err, color=color, fmt='o', alpha = 0.3, label = 'Background without ion contamination')

ax1.set_title('EPT, ' + direction + ', ' + date.strftime('%Y-%m-%d') + ', ' + averaging +  '  averaging, bg subtraction on', fontsize = 16)
ax1.set_ylabel(r'Flux [1/s cm$^{2}$ sr MeV]', fontsize = 16)
ax1.set_xlabel(r'Energy [MeV]', fontsize = 16)

ax1.tick_params(axis='y', which='major', labelsize=14)
ax1.legend(loc=0, fontsize = 12)
ax1.set_yscale('log')
ax1.set_xscale('log') 

plt.savefig(path +date_folder+ '/plots/EPT_ion_corr_spectra_comparison-'+direction+'-'+averaging+'.png', dpi=300, bbox_inches='tight')

In [ ]:
data_step